In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 44.9 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os, shutil
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_dataset_dir = '/content/drive/MyDrive/data'
base_dir = '/content/drive/MyDrive/data'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
conv_base = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256, 3))

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 256, 256, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 256, 256, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 256, 256, 3)  0           ['normalization[0][0

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 8, 8, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 81920)             0         
                                                                 
 dense (Dense)               (None, 1024)              83887104  
                                                                 
 dense_1 (Dense)             (None, 8)                 8200      
                                                                 
Total params: 87,944,875
Trainable params: 87,902,852
Non-trainable params: 42,023
_________________________________________________________________


In [ ]:
print('Это количество тренируемых весов '
            'перед замораживанием conv base:', len(model.trainable_weights))

Это количество тренируемых весов перед замораживанием conv base: 215


In [ ]:
conv_base.trainable = False

In [ ]:
print('Это количество тренируемых весов '
            'после заморозки conv base:', len(model.trainable_weights))

Это количество тренируемых весов после заморозки conv base: 4


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch = 10

train_datagen = ImageDataGenerator(rescale = 1. / 255, validation_split = 0.15)

test_datagen = ImageDataGenerator(rescale = 1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=batch,
    class_mode='categorical',
    shuffle=False)
label_map = (train_generator.class_indices)

validation_generator = train_datagen.flow_from_directory(
   train_dir,
   target_size = (256, 256),
   batch_size = batch,
   class_mode = 'categorical', 
   subset = "validation", 
   shuffle = False)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=batch,
    class_mode='categorical',
    shuffle=False)

label_map

Found 23219 images belonging to 8 classes.
Found 3479 images belonging to 8 classes.
Found 2585 images belonging to 8 classes.


{'Засветы': 0,
 'Малая часть упаковки видна на фото': 1,
 'Не читабилен текст': 2,
 'Нормальные': 3,
 'Размытые': 4,
 'Фото не в том режиме': 5,
 'Фото полки где 4-5 товара и видно что фоткали не основ товар': 6,
 'Фото ценника': 7}

In [ ]:
train_stop = 23219//batch
validation_stop = 3479//batch
test_stop = 2585//batch

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
%%time
history = model.fit(train_generator,
                    steps_per_epoch=train_stop,
                    epochs=10,
                    validation_data = validation_generator,
                    validation_steps = validation_stop,
                    verbose = 1)

Epoch 1/10
2321/2321 [==============================] - 6455s 3s/step - loss: 3.4263 - categorical_accuracy: 0.1382 - val_loss: 2.2715 - val_categorical_accuracy: 0.1228
Epoch 2/10
2321/2321 [==============================] - 2625s 1s/step - loss: 2.1120 - categorical_accuracy: 0.1052 - val_loss: 2.0795 - val_categorical_accuracy: 0.1248
Epoch 3/10
2321/2321 [==============================] - 2678s 1s/step - loss: 2.0827 - categorical_accuracy: 0.1089 - val_loss: 2.0880 - val_categorical_accuracy: 0.1245
Epoch 4/10
2321/2321 [==============================] - 2682s 1s/step - loss: 2.0797 - categorical_accuracy: 0.1127 - val_loss: 2.0795 - val_categorical_accuracy: 0.1228
Epoch 5/10
2321/2321 [==============================] - 2682s 1s/step - loss: 2.0817 - categorical_accuracy: 0.1190 - val_loss: 2.0795 - val_categorical_accuracy: 0.1228
Epoch 6/10
2321/2321 [==============================] - 2756s 1s/step - loss: 2.0818 - categorical_accuracy: 0.1176 - val_loss: 2.0794 - val_categoric

In [ ]:
test = model.evaluate(test_generator, steps=test_stop)

258/258 [==============================] - 735s 3s/step - loss: 2.0794 - categorical_accuracy: 0.1264


In [ ]:
import matplotlib.pyplot as plt

acc = history.history[tf.keras.metrics.CategoricalAccuracy()]
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('/content/drive/MyDrive/dataset/ENetB0notS.h5')

In [ ]:
import pandas as pd

In [ ]:
df_marks = pd.DataFrame({'x_test': test_generator.filepaths,
     'y_test': test_generator.labels})

x_test = df_marks.drop(['y_test'],axis=1)
y_test = df_marks['y_test']

In [ ]:
pred = model.predict(test_generator)

259/259 [==============================] - 204s 779ms/step


In [ ]:
predicted_classes = np.argmax(pred, axis = 1)
print(predicted_classes)

[3 3 3 ... 3 3 3]


In [ ]:
predicted_classes

array([3, 3, 3, ..., 3, 3, 3])

In [ ]:
df_pred = df_marks.join(predicted_classes)
df_pred

AttributeError: ignored

In [ ]:
df_pred.to_csv('/content/drive/MyDrive/dataset/file_pred.csv')

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
report = classification_report(y_test, predicted_classes)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       322
           1       0.00      0.00      0.00       325
           2       0.00      0.00      0.00       321
           3       0.13      1.00      0.22       326
           4       0.00      0.00      0.00       322
           5       0.00      0.00      0.00       321
           6       0.00      0.00      0.00       325
           7       0.00      0.00      0.00       323

    accuracy                           0.13      2585
   macro avg       0.02      0.12      0.03      2585
weighted avg       0.02      0.13      0.03      2585



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
